Members:
* Stephy Riega - 20171426
* Jesus Soto - 20172738
* Franco Caceres - 2016615

## 1. What is data splitting?

Data splitting consists in dividing the data *randomly* in at least two samples, training our models in one and testing their predict capacity in the other one. The first step to evaluate the prediction's performance of the model is data splitting. The proportion represented by each partitioned data varies depending of the researcher, but it is pretty common that the data has a proportion of 80%/20%. You are going to name "train" at the first split (which represents  the 80%) and "test" (20%) to the other one. It is important to take into account that this data splitting has to be done *randomly*. The second step is to perform the regression or regressions of different models in the training sample and, after that, perform the predict of your dependent variable ($Y_{test-estimated}$) on your test sample using your estimators ($\beta_{train}$) from the training sample. The third step is to estimate the $R^2$ and $MSE_{OFS}$ of the regressions by calculating the average of the squared difference between the observed ($Y_{test}$) and the estimaded ($Y_{test-estimated}$). With this, you can compared them with $R^2$  and $MSE$ into sample to know if your estimators perform better than in the sample data or which model perform better out of sample.

# 2. Replicate the PM1_Notebook1_Prediction_newdata (R and Python) JN but with restricted data:

As seen, Y  is the hourly wage of a worker and  X  is a vector of worker's characteristics, e.g., education, experience, gender. We focus on how to use job-relevant characteristics, such as education and experience, to best predict wages.

## Data


The data set we consider is from the March Supplement of the U.S. Current Population Survey, year 2015. We select white non-hispanic individuals, aged 25 to 64 years, and working more than 35 hours per week during at least 50 weeks of the year. We exclude self-employed workers; individuals living in group quarters; individuals in the military, agricultural or private household sectors; individuals with inconsistent reports on earnings and employment status; individuals with allocated or missing information in any of the variables used in the analysis; and individuals with hourly wage below  3 .

The variable of interest  Y  is the hourly wage rate constructed as the ratio of the annual earnings to the total number of hours worked, which is constructed in turn as the product of number of weeks worked and the usual number of hours worked per week. In our analysis, we also focus on single (never married) workers. The final sample is of size  **n=5150** .

## Data analysis

We start by loading the data set.

In [1]:
load("../data/wage2015_subsample_inference.Rdata")
dim(data)

Warning message in readChar(con, 5L, useBytes = TRUE):
"cannot open compressed file '../data/wage2015_subsample_inference.Rdata', probable reason 'No such file or directory'"

ERROR: Error in readChar(con, 5L, useBytes = TRUE): no se puede abrir la conexión


It's a data frame with 5150 observations with 20 variables. However,  **since we want only a subsample of the data that consists of the people who did not go to college**, we restrict the data set using the variables *shs* and *hsg* (which represents the people who ultimately went partially to high school or completed this level of education). 

In [ ]:
data <-subset(data, shs==1 | hsg==1)

dim (data) # less observations (1376) than before
head(data)

We are constructing the output variable $Y$ and the matrix $Z$ which includes the characteristics of workers that are given in the data.

In [ ]:
Y <- log(data$wage)
n <- length(Y)
Z <- data[-which(colnames(data) %in% c("wage","lwage"))]
p <- dim(Z)[2]

cat("Number of observation:", n, '\n')
cat( "Number of raw regressors:", p)

For the outcome variable *wage* and a subset of the raw regressors, we calculate the empirical mean to get familiar with the data.

In [ ]:
library(xtable)
options(xtable.floating = FALSE)
options(xtable.timestamp = "")

In [ ]:
library(xtable)
Z_subset <- data[which(colnames(data) %in% c("lwage","sex","shs","hsg","scl","clg","ad","mw","so","we","ne","exp1"))]
table <- matrix(0, 12, 1)
table[1:12,1]   <- as.numeric(lapply(Z_subset,mean))
rownames(table) <- c("Log Wage","Sex","Some High School","High School Graduate","Some College","College Graduate", "Advanced Degree","Midwest","South","West","Northeast","Experience")
colnames(table) <- c("Sample mean")
tab<- xtable(table, digits = 2)
tab

E.g., the share of female workers in our sample is ~32% ($sex=1$ if female).

Alternatively, we can also print the table as latex.

In [ ]:
print(tab, type="latex")

## Prediction Question

We will construct a prediction rule for hourly wage $Y$, which depends linearly on job-relevant characteristics $X$:

\begin{equation}\label{decompose}
Y = \beta'X+ \epsilon.
\end{equation}

With this, we will:

* Predict wages  using various characteristics of workers.

* Assess the predictive performance using the (adjusted) sample MSE, the (adjusted) sample $R^2$ and the out-of-sample MSE and $R^2$.


We employ two different specifications for prediction:


1. Basic Model:   $X$ consists of a set of raw regressors (e.g. gender, experience, education indicators,  occupation and industry indicators, regional indicators). That is,  sex + exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2.


2. Flexible Model:  $X$ consists of all raw regressors from the basic model (excepto for sex) plus transformations (e.g., ${exp}^2$, ${exp}^3$ and ${exp}^4$) squared.That is, $(exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)^2$


Using the **Flexible Model**, enables us to approximate the real relationship by a more complex regression model and therefore to reduce the bias. The **Flexible Model** increases the range of potential shapes of the estimated regression function. In general, flexible models often deliver good prediction accuracy but give models which are harder to interpret. *Also, with flexible models we have the trade off between bias and variance, making it more propense to overfitting.*

Now, let us fit both models to our data by running ordinary least squares (ols):

In [ ]:
# 1. basic model
basic <- lwage~ (sex + exp1 + shs + hsg+ scl + clg + mw + so + we +occ2+ind2)
regbasic <- lm(basic, data=data)
regbasic # estimated coefficients
cat( "Number of regressors in the basic model:",length(regbasic$coef), '\n') # number of regressors in the Basic Model


##### Note that the basic model consists of $51$ regressors.

In [ ]:
# 2. flexible model
flex <- lwage ~ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2

regflex <- lm(flex, data=data)
regflex # estimated coefficients
cat( "Number of regressors in the flexible model:",length(regflex$coef)) # number of regressors in the Flexible Model

Note that the flexible model consists of $979$ regressors.

Try lasso next

In [ ]:
#Lasso model
library(hdm)
flex <- lwage ~ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2
lassoreg<- rlasso(flex, data=data)

sumlasso<- summary(lassoreg)


Now, we can evaluate the performance of both models based on the (adjusted) $R^2_{sample}$ and the (adjusted) $MSE_{sample}$:

In [ ]:
# Assess the predictive performance

sumbasic <- summary(regbasic)
sumflex <- summary(regflex)

#  R-squared 
R2.1 <- sumbasic$r.squared
cat("R-squared for the basic model: ", R2.1, "\n")
R2.adj1 <- sumbasic$adj.r.squared
cat("adjusted R-squared for the basic model: ", R2.adj1, "\n")

R2.2 <- sumflex$r.squared
cat("R-squared for the flexible model: ", R2.2, "\n")
R2.adj2 <- sumflex$adj.r.squared
cat("adjusted R-squared for the flexible model: ", R2.adj2, "\n")

R2.L <- sumlasso$r.squared
cat("R-squared for the lasso with flexible model: ", R2.L, "\n")
R2.adjL <- sumlasso$adj.r.squared
cat("adjusted R-squared for the flexible model: ", R2.adjL, "\n")

In [ ]:
# calculating the MSE
MSE1 <- mean(sumbasic$res^2)
cat("MSE for the basic model: ", MSE1, "\n")
p1 <- sumbasic$df[1] # number of regressors
MSE.adj1 <- (n/(n-p1))*MSE1
cat("adjusted MSE for the basic model: ", MSE.adj1, "\n")

MSE2 <-mean(sumflex$res^2)
cat("MSE for the flexible model: ", MSE2, "\n")
p2 <- sumflex$df[1]
MSE.adj2 <- (n/(n-p2))*MSE2
cat("adjusted MSE for the flexible model: ", MSE.adj2, "\n")


MSEL <-mean(sumlasso$res^2)
cat("MSE for the lasso flexible model: ", MSEL, "\n")
pL <- length(sumlasso$coef)
MSE.adjL <- (n/(n-pL))*MSEL
cat("adjusted MSE for the lasso flexible model: ", MSE.adjL, "\n")
cat("adjusted MSE for the partialing out lasso flexible model: ", MSE.adjL2, "\n")

In [ ]:
library(xtable)
table <- matrix(0, 3, 5)
table[1,1:5]   <- c(p1,R2.1,MSE1,R2.adj1,MSE.adj1)
table[2,1:5]   <- c(p2,R2.2,MSE2,R2.adj2,MSE.adj2)
table[3,1:5]   <- c(pL,R2.L,MSEL,R2.adjL,MSE.adjL)
colnames(table)<- c("p","$R^2_{sample}$","$MSE_{sample}$","$R^2_{adjusted}$", "$MSE_{adjusted}$")
rownames(table)<- c("basic reg","flexible reg", "lasso flex")
tab<- xtable(table, digits =c(0,0,2,2,2,2))
print(tab,type="latex") # type="latex" for printing table in LaTeX
tab

Considering all measures above, the flexible model performs slightly better than the other models. 

One procedure to circumvent this issue is to use **data splitting** that is described and applied in the following.

## Data Splitting

Measure the prediction quality of the two models via data splitting:

- Randomly split the data into one training sample and one testing sample. Here we just use a simple method (stratified splitting is a more sophiscticated version of splitting that we can consider).
- Use the training sample for estimating the parameters of the Basic Model and the Flexible Model.
- Use the testing sample for evaluation. Predict the $\mathtt{wage}$  of every observation in the testing sample based on the estimated parameters in the training sample.
- Calculate the Mean Squared Prediction Error $MSE_{test}$ based on the testing sample for both prediction models. 

In [ ]:
n

In [ ]:
#splitting the data

set.seed(1) # to make the results replicable (generating random numbers)
random <- sample(1:n, floor(n*4/5))
length(random)

In [ ]:
# draw (4/5)*n random numbers from 1 to n without replacing them
train <- data[random,] # training sample
test <- data[-random,] # testing sample
dim(test)

In [ ]:
# basic model
# estimating the parameters in the training sample
regbasic <- lm(basic, data=train)
# calculating the out-of-sample MSE
trainregbasic <- predict(regbasic, newdata=test)

y.test <- log(test$wage)
MSE.test1 <- sum((y.test-trainregbasic)^2)/length(y.test)
R2.test1<- 1- MSE.test1/var(y.test)

cat("Test MSE for the basic model: ", MSE.test1, " ")

cat("Test R2 for the basic model: ", R2.test1)

In the basic model, the $MSE_{test}$ is quite closed to the $MSE_{sample}$.

In [ ]:
# flexible model
# estimating the parameters
#options(warn=-1)
regflex <- lm(flex, data=train)

# calculating the out-of-sample MSE
trainregflex<- predict(regflex, newdata=test)

y.test <- log(test$wage)
MSE.test2 <- sum((y.test-trainregflex)^2)/length(y.test)
R2.test2<- 1- MSE.test2/var(y.test)

cat("Test MSE for the flexible model: ", MSE.test2, " ")

cat("Test R2 for the flexible model: ", R2.test2)

In the flexible model, the discrepancy between the $MSE_{test}$ and the $MSE_{sample}$ is not large.

It is worth to notice that the $MSE_{test}$ vary across different data splits. Hence, it is a good idea average the out-of-sample MSE over different data splits to get valid results.

Nevertheless, we observe that, based on the out-of-sample $MSE$, the basic model using ols regression performs is about as well (or slightly better) than the flexible model. 


Next, let us use lasso regression in the flexible model instead of ols regression. Lasso (*least absolute shrinkage and selection operator*) is a penalized regression method that can be used to reduce the complexity of a regression model when the number of regressors $p$ is relatively large in relation to $n$. 

Note that the out-of-sample $MSE$ on the test sample can be computed for any other black-box prediction method as well. Thus, let us finally compare the performance of lasso regression in the flexible model to ols regression.

In [ ]:
# flexible model using lasso

# estimating the parameters
library(hdm)
reglasso <- rlasso(flex, data=train, post=FALSE)

# calculating the out-of-sample MSE
trainreglasso<- predict(reglasso, newdata=test)
MSE.lasso <- sum((y.test-trainreglasso)^2)/length(y.test)
R2.lasso<- 1- MSE.lasso/var(y.test)


cat("Test MSE for the lasso on flexible model: ", MSE.lasso, " ")

cat("Test R2 for the lasso flexible model: ", R2.lasso)

Finally, let us summarize the results:

In [ ]:
table2 <- matrix(0, 3,2)
table2[1,1]   <- MSE.test1
table2[2,1]   <- MSE.test2
table2[3,1]   <- MSE.lasso
table2[1,2]   <- R2.test1
table2[2,2]   <- R2.test2
table2[3,2]   <- R2.lasso

rownames(table2)<- c("basic reg","flexible reg","lasso regression")
colnames(table2)<- c("$MSE_{test}$", "$R^2_{test}$")
tab2 <- xtable(table2, digits =3)
tab2

We can see now that the lasso regression is slightly better at predicting the data than was before using OLS. However, we can see that, overall, **the basic model is better at predicting the test set.**

In [ ]:
print(tab2,type="latex") # type="latex" for printing table in LaTeX

## 2. In addition Do two cases of Partialling-Out using lasso. Remember that we want to find the beta associated with sex.

### Example 1: Matrix W = 'exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2'

In [ ]:
library(hdm)
# For the basic model
basic.y <- lwage ~ (exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2) # Model for Y
basic.d <- sex ~ (exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2) # Modelo for D

# Residuals
basic_rY <- rlasso(basic.y, data = data)$res
basic_rD <- rlasso(basic.d, data = data)$res

# regression of Y on D after partialling-out the effect of W
basic_partial.fit <- lm(basic_rY ~ basic_rD)
basic_partial.est <- summary(basic_partial.fit)$coef[2,1]

sum_basiclasso <- summary(basic_partial.fit)

cat("Coefficient for D via partialling-out",basic_partial.est)


The use of the lasso for partialling-out the basic regressions shows us that the gender gap is of $9.066\%$, which is different from the $7\%$ gap found int the basic regression using OLS. Probably because lasso supresses some of the regressors but not all of them, and because partialling out controls for the covariates in the both regressions.

### Example 2: Matrix W = '(exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2'

In [ ]:
library(hdm)
# For the flexible model
flex.y <- lwage ~ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2 # Model for Y
flex.d <- sex ~ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2 # Modelo for D

# Residuals
flex_rY <- rlasso(flex.y, data = data)$res
flex_rD <- rlasso(flex.d, data = data)$res

# regression of Y on D after partialling-out the effect of W
flex_partial.fit <- lm(flex_rY ~ flex_rD)
flex_partial.est <- summary(flex_partial.fit)$coef[2,1]

sum_flexlasso <- summary(flex_partial.fit)

cat("Coefficient for D via partialling-out",flex_partial.est)


The use of the lasso for partialling-out the basic regressions shows us that the gender gap is of $8.14\%$, which is different from the $7\%$ gap found int the basic regression using OLS but **pretty similar to the $9.066\%$ found on the basic regression model with partialling-out.**